# How to Echo State
## By Kevin G. R. Pergher
Um passo a passo de como gerar e aplicar redes ESN em séries temporais

### (0) Import das bibliotecas a serem utilizadas & funções úteis

In [1]:
import numpy
from scipy import linalg
import matplotlib.pyplot as plt

Funções auxiliares a serem utilizadas durante o processo

In [2]:
# Garante o formato das colunas para arrays unidimensionais
def reshape(df):
    if len(df.shape) < 2:
        return numpy.reshape(df, (df.shape[0], 1))
    else:
        return df

### (I) Extração e tratamento dos dados

Nesse momento, carregamos o **dataset** de interesse (série temporal). Por conta da biblioteca Numpy é necessário garantir com a função **reshape()** o formato do array (há ausência de valor de colunas quando a série é unidimensional).

Em nossa aplicações, **K** é o número de colunas/variáveis de input a serem utilizadas (i.e. $X_0$, $X_1$, ..., $X_{K-1}$). Como em nossa aplicações utilizamos a série para prever "ela mesma no futuro", o número de colunas/variáveis de output **L** equivale ao de input.

### CORREÇÃO
**K** não é, nem precisa ser, igual ou equivalente a **L**.
**K** é o sinal de entrada, portanto **K** é a tua janela de entrada. Na verdade, em todos os casos que vamos utilizar a ESN, **K** é muito diferente de **L**. Vamos lá: no caso da Mackey-Glass, **K** é a janela de entrada (maior que 1) e **L** é o valor alvo, é igual a 1. Precisa resolver isso daqui o quanto antes!

In [3]:
dataset = reshape(numpy.loadtxt('NormHenon.txt', delimiter = ","))

K = dataset.shape[1]
L = K

print(dataset.shape)
dataset

(1000000, 2)


array([[-0.50345012, -0.50072158],
       [ 1.1035704 , -0.50072158],
       [-0.97664434,  1.10224828],
       ...,
       [ 1.57941742, -0.36847491],
       [-1.68841231,  1.59306695],
       [-1.0234445 , -1.6818749 ]])

Definimos algumas variáveis relativas ao processamento a seguir, **steps** é o número de passos adiante que pretendemos prever (steps ahead), **train_length** é o tamanho (número de observações) da janela de treino e **test_length** é o tamanho (número de observações) da janela de teste/validação.

A variável **transiente** equivale aos dados descartados do período de teste.

In [4]:
steps = 1

train_length = 3000
test_length = 1000

transiente = 500

Definimos **train_dataset** como a janela de treino e **test_dataset** como a janela de teste. Definimos **train_target** como a janela de treino deslocada $n$ steps à frente para fins de aplicação da regressão de Ridge e **test_dataset** como a janela de teste deslocada $n$ steps à frente para fins de medida de erro do modelo. As janelas de teste se iniciam após as janelas de treino (incluindo o target).

In [5]:
train_dataset = dataset[0 : train_length, :]
train_target = dataset[None, steps + transiente : train_length + steps, :]

test_dataset = dataset[train_length : train_length + test_length, :]
test_target = dataset[None, train_length + steps : train_length + test_length + steps, :]

In [6]:
print("Tamanho - Treino: ", train_dataset.shape)
print("Tamanho - Target/Treino: ", train_target.shape)

Tamanho - Treino:  (3000, 2)
Tamanho - Target/Treino:  (1, 2500, 2)


In [7]:
train_dataset

array([[-0.50345012, -0.50072158],
       [ 1.1035704 , -0.50072158],
       [-0.97664434,  1.10224828],
       ...,
       [ 1.53127093, -0.78401452],
       [-1.70194019,  1.54293806],
       [-1.08426284, -1.69490931]])

In [8]:
train_target

array([[[ 0.54293852,  0.30451498],
        [ 0.22823854,  0.53618457],
        [ 0.83778263,  0.22314368],
        ...,
        [-1.70194019,  1.54293806],
        [-1.08426284, -1.69490931],
        [-0.08058896, -1.08515286]]])

In [9]:
print("Tamanho - Teste: ", test_dataset.shape)
print("Tamanho - Target/Teste: ", test_target.shape)

Tamanho - Teste:  (1000, 2)
Tamanho - Target/Teste:  (1, 1000, 2)


In [10]:
test_dataset

array([[-0.08058896, -1.08515286],
       [ 0.59978986, -0.0820576 ],
       [ 0.00855218,  0.59305357],
       ...,
       [ 0.86624938,  0.12923607],
       [-0.41309421,  0.86113568],
       [ 1.6000105 , -0.41079576]])

In [11]:
test_target

array([[[ 0.59978986, -0.0820576 ],
        [ 0.00855218,  0.59305357],
        [ 1.16839323,  0.00592743],
        ...,
        [-0.41309421,  0.86113568],
        [ 1.6000105 , -0.41079576],
        [-1.72740329,  1.61454086]]])

### (II) Criação do reservatório

Definimos o número de neurônios na camada oculta do reservatório, chamado **N (neurons)**, e a taxa de vazamento espectral **leaking_rate** (entre 0 e 1). 

O **input_scaling** define a escala dos valores da matriz de pesos de entrada e **spectral_radius** define o raio espectral desejado para a matriz de pesos do reservatório.

Define-se a semente do gerador para reprodução do experimento.

In [29]:
N = 10
leaking_rate = 0.3
input_scaling = 1
spectral_radius = 1.25

numpy.random.seed(42)

São criadas duas matrizes aleatórias, através de uma distribuição uniforme de valores entre $-0,5$ e $0,5$. A matriz **W_input** equivale aos pesos de conexão entre a entrada (input) e o reservoir e têm dimensões $N\ x\ (K\ +\ 1)$ pois considera o input e o bias, a matriz é escalada por **input_scaling**. 

A matriz **W_reservoir** é definida como a matriz de pesos do reservatório (inter e intra neurônios) e têm dimensão $N\ x\ N$.

In [30]:
W_input = (numpy.random.rand(N, 1 + K) - 0.5) * input_scaling
W_reservoir = numpy.random.rand(N, N) - 0.5 

In [21]:
W_input

array([[-0.12545988,  0.45071431,  0.23199394],
       [ 0.09865848, -0.34398136, -0.34400548],
       [-0.44191639,  0.36617615,  0.10111501],
       [ 0.20807258, -0.47941551,  0.46990985],
       [ 0.33244264, -0.28766089, -0.31817503],
       [-0.31659549, -0.19575776,  0.02475643],
       [-0.06805498, -0.20877086,  0.11185289],
       [-0.36050614, -0.20785535, -0.13363816],
       [-0.04393002,  0.28517596, -0.30032622],
       [ 0.01423444,  0.09241457, -0.45354959],
       [ 0.10754485, -0.32947588, -0.43494841],
       [ 0.44888554,  0.46563203,  0.30839735],
       [-0.19538623, -0.40232789,  0.18423303],
       [-0.05984751, -0.37796177, -0.00482309],
       [-0.46561148,  0.4093204 , -0.24122002],
       [ 0.16252228, -0.18828892,  0.02006802],
       [ 0.04671028, -0.31514554,  0.46958463],
       [ 0.27513282,  0.43949894,  0.39482735],
       [ 0.09789998,  0.42187424, -0.4115075 ],
       [-0.30401714, -0.45477271, -0.17466967],
       [-0.11132271, -0.22865097,  0.328

In [22]:
W_reservoir

array([[ 4.08265886e-01, -2.60438109e-01, -3.55105128e-01, ...,
         3.97110260e-01,  3.87086424e-01,  2.79875546e-01],
       [ 1.42031646e-01, -4.15860035e-01, -3.38371286e-01, ...,
         4.66654819e-01,  4.63619977e-01,  3.53009455e-01],
       [-2.05551108e-01, -1.14902271e-01,  3.51136672e-01, ...,
        -2.84178973e-01,  1.22890476e-01, -4.14652535e-01],
       ...,
       [ 3.27518922e-01,  2.64527795e-01,  7.35289515e-02, ...,
        -9.24569690e-02, -9.28935109e-02, -4.33990156e-01],
       [-1.51179466e-01, -3.89001901e-01,  3.08235210e-01, ...,
        -3.30457718e-03, -2.15725907e-01, -3.66171637e-01],
       [ 1.29557697e-01, -4.45667965e-01,  2.48645234e-01, ...,
         4.01145710e-04, -6.72708640e-02, -4.23003948e-02]])

Obtemos o maior autovalor da matriz de pesos de reservatório **rhoReservoir**. Dividimos a matriz por este escalar e multiplicamos pelo raio espectral desejado.

In [31]:
rhoReservoir = numpy.max(numpy.abs(linalg.eig(W_reservoir)[0]))
W_reservoir *= spectral_radius / rhoReservoir

In [17]:
#W_reservoir[numpy.random.rand(N, N) < 0.9] = 0

### (III) Matriz de estados e harvesting

Criamos a matriz de estados **state_matrix** inicialmente zerada e de dimensões $(1\ +\ K\ +\ N)\ x\ (Janela\ de\ treino - Descarte)$. Cada linha da matriz armazena o bias, o input(naquele ponto) e estado de cada um dos neurônios no decorrer da janela de treino, à exceção do período do transiente que é descartado.

In [32]:
state_matrix = numpy.zeros((1 + K + N, train_length - transiente))
state_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Passamos ao processo de harvesting, a matriz é atualizada conforme a *equação de atualização dos estados* escolhida. A variável auxiliar **state** armazena os valores de interesse e repassa para a matriz de estados.

Conforme o processo é iterado nas $n$ observações da janela de treino, a matriz de estados recebe igualmente o bias (por definição é 1) e o input relativo a $n$. Valores calculados antes do ponto de corte do transiente não são adicionados na matriz de estados final.

In [33]:
state = numpy.zeros((N, 1))
#for n in range(train_dataset.shape[0]):
n = 0
u = train_dataset[n,:]
state = (1 - leaking_rate)*state + leaking_rate*numpy.tanh(numpy.dot(W_input, reshape(numpy.hstack((1, u)))) + numpy.dot(W_reservoir, state))
#if n >= transiente:
#    state_matrix[:, n - transiente] = numpy.hstack((1, u, state[:,0]))
state_matrix[:, n] = numpy.hstack((1, u, state[:,0]))

state_matrix

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.50345012,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.50072158,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.0560187 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01113151,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05771484,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
state_matrix.shape

(13, 2500)

In [40]:
y = numpy.array([1, 2, 3])
y.shape
x = numpy.arange(10, 31).reshape((7, 3))
x.shape
numpy.dot(y, x.T)

array([ 68,  86, 104, 122, 140, 158, 176])

### (IV) Regressão de Ridge

Aplicamos a regressão de Tikhonov para estimar a melhor matrix de pesos de saída **W_output** de dimensão $L\ x\ (K\ +\ 1\ +\ N)$, isso é, aquela à qual o erro com relação ao target esperado (conjunto de trainning) é o menor possível.

In [ ]:
reg = 1e-8
W_output = linalg.solve(numpy.dot(state_matrix, state_matrix.T) + reg * numpy.eye(1 + K + N), 
    numpy.dot(state_matrix, train_target.T) ).T[0]
W_output

In [ ]:
W_output.shape

### (V) Dataset de teste

Todo o processe de criação das matrizes é "reiniciado", perceba-se que o gerador não é inializado de novo (pode ser inicializado com outro valor), são criadas novas matrizes de pesos de entreda e de reservatório (nas mesmas condições que as anteriores, mas não identicas). 

O processo de harvesting é iniciado do zero, desta vez considerando apenas os dados de teste/validação, dessa forma teremos novos estados, conforme os dados apresentados e a equação de atualização.

### Dúvida/Constatação I

Havia entendido que poderiamos gerar novas matrizes de peso (input e reservatório) e algoritmo funcionaria apenas com algum erro, mas pelo que vejo, o W_{output} tem serventia na predição somente quando as matrizes de pesos de input e reservatorio são as mesmas! (tende mudar abaixo as matrizes -> A curva tem o formato certo, mas os valores não coincidem!)

Assim, **toda vez que realizarmos o trainning é necessário guardar as 3 matrizes para o teste (W_{input}, W_{reservoir} e W_{output})?**

Sim.

In [ ]:
#W_input = (numpy.random.rand(N, 1 + K) - 0.5) * input_scaling
#W_reservoir = numpy.random.rand(N, N) - 0.5 
#W_input

In [ ]:
#rhoReservoir = numpy.max(numpy.abs(scipy.linalg.eig(W_reservoir)[0]))
#W_reservoir *= spectral_radius / rhoReservoir

### Truque

Como não faz sentido o uso de transiente no conjunto de teste (ou ele invade o conjunto de treino ou desperdiça dados do conjunto de teste), fiz uso desse truque que vi em outra implementação de ESN, o segredo é iniciar o estado com o último estado obtido na matriz de estados do conjunto de treino!

Dessa forma é necessário guardar esse vetor, mas elimina o uso de transiente e o resultado fica igualmente excelente.

### OBSERVAÇÃO
O uso dos estados transientes só faz sentido na etapa de treino pois lá tu estás usando uma função de aprendizagem e descarta o que seriam estados "sub-ótimos". Depois do treino realizado, tu vai entrar no modo de predição de 1, ou mais, passos adiante. Mas isso não deveria ser um truque. São duas funções **bem** distintas: uma para aprendizagem e outra para uso do modelo. A primeira depende de u, X, Win, etc. e resulta na criação/estimação de Wout, enquanto que a outra depende só de u, X, Win, W e Wout. Na segunda função, a saída é um vetor y estimado de dimensão L.

In [ ]:
state = reshape(state_matrix[K + 1::,-1])
state.shape

In [ ]:
state_matrix = numpy.zeros((1 + K + N, test_length))
state_matrix

In [ ]:
for n in range(test_dataset.shape[0]):
    u = test_dataset[n,:]
    #state = (1 - leaking_rate)*state + leaking_rate*numpy.tanh(numpy.dot(W_input, numpy.vstack((1, u))) + numpy.dot(W_reservoir, state))
    #state_matrix[:, n] = numpy.vstack((1, u, state))[:, 0]
    state = (1 - leaking_rate)*state + leaking_rate*numpy.tanh(numpy.dot(W_input, reshape(numpy.hstack((1, u)))) + numpy.dot(W_reservoir, state))
    state_matrix[:, n] = numpy.hstack((1, u, state[:,0]))
    

state_matrix

In [ ]:
state_matrix.shape

### (VI) Predição

Por fim, utilizamos a mesma matriz de pesos de saída **W_output** otimizada no período de trainning e aplicamo-la sobre os estados obtidos via processo de harvesting para o dataset de teste. Assim obtemos a predição dos valores $n$ steps a frente que pode ser comparado com o target (esperado) do conjunto de teste.

### OBSERVAÇÃO

Os comandos abaixo deveriam estar contidos na função acima, pois não entendi o porquê de fazer de forma separada, dado que o objetivo são os valores preditos (1 ou mais passos adiante).

In [ ]:
predict = numpy.dot(W_output, state_matrix)
predict.shape

In [ ]:
test_target.shape

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#ax.scatter(predict[0,:], predict[1,:], predict[2,:], color = "blue")
#ax.scatter(test_target[0, :, 0], test_target[0, :, 1], test_target[0, :, 2], color = "red")
#plt.show()

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(test_target[0, :, 0], test_target[0, :, 1], color = "blue", linewidth = 2, label = "Expected")
plt.scatter(predict[0,:], predict[1,:], color = "red", linewidth = 1, label = "Predict")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
test_target[0].T

In [ ]:
predict

In [ ]:
mean_squared_error(test_target[0].T, predict)

In [ ]:
mean_squared_error(test_target[0].T, predict, squared = False)

In [ ]:
mean_absolute_percentage_error(test_target[0].T, predict)